In [5]:
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [13]:
data=pd.read_csv('zomato.csv', encoding='latin1')
data.describe()

,Restaurant ID,Country Code,Longitude,Latitude,Average Cost for two,Price range,Aggregate rating,Votes
count,9.551000e+03,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000,9551.000000
mean,9.051128e+06,18.365616,64.126574,25.854381,1199.210763,1.804837,2.666370,156.909748
std,8.791521e+06,56.750546,41.467058,11.007935,16121.183073,0.905609,1.516378,430.169145
min,5.300000e+01,1.000000,-157.948486,-41.330428,0.000000,1.000000,0.000000,0.000000
25%,3.019625e+05,1.000000,77.081343,28.478713,250.000000,1.000000,2.500000,5.000000
50%,6.004089e+06,1.000000,77.191964,28.570469,400.000000,2.000000,3.200000,31.000000
75%,1.835229e+07,1.000000,77.282006,28.642758,700.000000,2.000000,3.700000,131.000000
max,1.850065e+07,216.000000,174.832089,55.976980,800000.000000,4.000000,4.900000,10934.000000


In [23]:
is_indian_restaurants=data['Country Code']==1
is_indian_restaurants.describe()

count     9551
unique       2
top       True
freq      8652
Name: Country Code, dtype: object

In [27]:
indian = data[is_indian_restaurants]
indian.shape

(8652, 21)

In [36]:
is_delhi = data['City']=='New Delhi'
delhi_restaurants = indian[is_delhi]
delhi_restaurants.shape

C:\Users\taghos\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


(5473, 21)

In [34]:
data.columns

Index(['Restaurant ID', 'Restaurant Name', 'Country Code', 'City', 'Address',
       'Locality', 'Locality Verbose', 'Longitude', 'Latitude', 'Cuisines',
       'Average Cost for two', 'Currency', 'Has Table booking',
       'Has Online delivery', 'Is delivering now', 'Switch to order menu',
       'Price range', 'Aggregate rating', 'Rating color', 'Rating text',
       'Votes'],
      dtype='object')

In [41]:
#Extracting only the relevant columns
data_minimal=delhi_restaurants[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Locality', 'Aggregate rating']]
data_minimal

,Restaurant Name,Cuisines,Average Cost for two,Locality,Aggregate rating
2560,Food Cloud,Cuisine Varies,500,Aaya Nagar,0.0
2561,Burger.in,Fast Food,350,Adchini,3.2
2562,Days of the Raj,"North Indian, Seafood, Continental",1500,Adchini,3.4
2563,Dilli Ka Dhaba,"South Indian, North Indian",500,Adchini,2.6
2564,Govardhan,"South Indian, North Indian, Chinese",500,Adchini,3.4
2565,Mezbaan Grills,Mughlai,400,Adchini,3.1
2566,Say Cheese,Fast Food,400,Adchini,2.7
2567,Southy,South Indian,450,Adchini,2.6
2568,Monosoz,Pizza,300,Adchini,3.7
2569,Waves,"North Indian, Chinese",1500,Adchini,3.5


In [43]:
data_minimal['Locality'].value_counts(dropna=False)

Connaught Place               122
Rajouri Garden                 99
Shahdara                       87
Defence Colony                 86
Pitampura                      85
                             ... 
Centaur Hotel, Aerocity         1
Mansingh Road                   1
JW Marriott New Delhi           1
Tilak Marg                      1
Piccadily Hotel, Janakpuri      1
Name: Locality, Length: 254, dtype: int64

In [50]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

data_sample=[]
def restaurant_recommendation(location, title):
    global data_sample       
    global cosine_sim
    global sim_scores
    global tfidf_matrix
    global corpus_index
    global feature
    global rest_indices
    global idx
    
    data_sample=data_minimal.loc[data_minimal['Locality']==location]
    data_sample.reset_index(level=0, inplace=True)
    data_sample['Split']="X"
    for i in range(0, data_sample.index[-1]):
        split_data=re.split(r'[,]', data_sample['Cuisines'][i])
        for k,l in enumerate(split_data):
            split_data[k]=(split_data[k].replace(" ", ""))
        split_data=' '.join(split_data[:])
        data_sample['Split'].iloc[i]=split_data
    tfidf=TfidfVectorizer(stop_words='english')
    data_sample['Split']=data_sample['Split'].fillna('')
    tfidf_matrix=tfidf.fit_transform(data_sample['Split'])
    tfidf_matrix.shape
    feature=tfidf.get_feature_names()
    
    cosine_sim=linear_kernel(tfidf_matrix, tfidf_matrix)
    corpus_index=[n for n in data_sample['Split']]
    indices=pd.Series(data_sample.index, index=data_sample['Restaurant Name']).drop_duplicates()
    idx=indices[title]
    sim_scores=[]
    for i,j in enumerate(cosine_sim[idx]):
        k=data_sample['Aggregate rating'].iloc[i]
        if j != 0 :
            sim_scores.append((i,j,k))
    sim_scores = sorted(sim_scores, key=lambda x: (x[1],x[2]) , reverse=True)
    sim_scores = sim_scores[0:10]
    
    rest_indices=[i[0] for i in sim_scores]
    data_x=data_sample[['Restaurant Name','Aggregate rating']].iloc[rest_indices]
    data_x['Cosine Similarity']=0
    for i,j in enumerate(sim_scores):
        data_x['Cosine Similarity'].iloc[i]=round(sim_scores[i][1],2)
   
    return data_x

restaurant_recommendation('Connaught Place', 'KFC')

C:\Users\taghos\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\taghos\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\taghos\AppData\Local\Continuum\anaconda3\envs\myenv\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in 

,Restaurant Name,Aggregate rating,Cosine Similarity
44,KFC,3.7,1.00
107,Johnny Rockets,4.0,0.78
83,The Rolling Joint,3.9,0.62
8,Indian Coffee House,3.3,0.62
58,Nizam's Kathi Kabab,3.8,0.54
75,Subway,3.5,0.54
108,Odeon Social,4.1,0.49
93,38 Barracks,4.4,0.48
86,Warehouse Cafe,3.7,0.45
82,The Luggage Room Kitchen And Bar,3.5,0.44
